In [15]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:41581")
client

Client Scheduler: tcp://127.0.0.1:41581 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.24 GB


In [16]:
import pandas as pd
import joblib
import gcsfs

#Dask
import dask.dataframe as dd
from dask_ml.cluster import SpectralClustering
from dask_ml.cluster import KMeans
from dask_ml.model_selection import train_test_split, GridSearchCV, IncrementalSearchCV

#Sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import svm, linear_model, tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

#Local Files
import src.features_engineering as fte
import src.clustering as cl
import src.supervised_learning as sl
import src.mongodb_database as mdb

In [17]:
ratings = dd.read_csv('./input/ratings_small.csv')
genres_dummies = pd.read_csv('./input/genres_dummies.csv')

### Features Engineering Pipeline

In [18]:
%%time
ratings = (ratings.pipe(fte.addUserFeatures)
           .pipe(fte.addMoviesFeatures)
           .pipe(fte.filterbyRatingsAmount, min_rt=100, max_rt=1000)
           .pipe(fte.addWeekdayColumns)
           .pipe(fte.addGenresDummies, genres_dummies=genres_dummies)
           #.pipe(fte.popularityNormalizer)
           #.pipe(fte.ratingsNormalizer)
          )

CPU times: user 533 ms, sys: 10.1 ms, total: 543 ms
Wall time: 1.81 s


/home/castares/.local/lib/python3.6/site-packages/dask/dataframe/multi.py:1055: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


In [19]:
ratings.columns

Index(['userId', 'movieId', 'GT', 'timestamp', 'user_rt_count', 'user_rt_mean',
       'movie_rt_mean', 'popularity', 'weekday', 'weekday_6', 'weekday_2',
       'weekday_3', 'weekday_0', 'weekday_1', 'weekday_5', 'weekday_4', 'id',
       'Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV',
       'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'],
      dtype='object')

# __Clustering__

In [20]:
users_genres = cl.dataScaling(cl.userGenresMatrix(ratings, genres_dummies))
users_genres_da = users_genres.to_dask_array(lengths=True)
users_genres.head()

,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,Drama,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
userId,,,,,,,,,,,,,,,,,,,,,
4,0.259259,0.183333,0.230769,0.0,0.0,0.0,0.211864,0.225352,0.076923,0.222222,...,0.266667,0.315789,0.0,0.0,0.0,0.0,0.254386,0.0,0.176471,0.000000
8,0.061728,0.033333,0.076923,0.0,0.0,0.0,0.076271,0.028169,0.230769,0.051587,...,0.077778,0.052632,0.0,0.0,0.0,0.0,0.026316,0.0,0.000000,0.090909
17,0.283951,0.350000,0.307692,0.0,0.0,0.0,0.245763,0.366197,0.076923,0.273810,...,0.200000,0.333333,0.0,0.5,0.0,0.0,0.324561,0.0,0.235294,0.545455
19,0.580247,0.600000,0.538462,0.0,0.0,0.0,0.491525,0.690141,0.461538,0.527778,...,0.500000,0.491228,0.0,0.0,0.0,0.0,0.552632,0.0,0.352941,0.272727
21,0.135802,0.133333,0.230769,0.0,0.0,0.0,0.110169,0.225352,0.153846,0.142857,...,0.177778,0.140351,0.0,0.0,0.0,0.0,0.140351,0.0,0.058824,0.090909


In [21]:
%%time
#Spectral Clustering
clusters_number = 4
spcl = SpectralClustering(n_clusters=clusters_number, affinity='polynomial', n_jobs=-1)
with joblib.parallel_backend('dask'):
    clusters = spcl.fit_predict(users_genres_da)
clusters_index = cl.getClustersIndex(clusters, users_genres)
#clusters_index.to_csv('./output/clusters-index/clusters-index-spcl-poly-4-*.csv')
clusters_index.compute()['cluster'].value_counts()

CPU times: user 26 s, sys: 1.91 s, total: 27.9 s
Wall time: 1min 49s


2    98
0    75
1    45
3    30
Name: cluster, dtype: int64

In [22]:
ratings = ratings.merge(clusters_index, left_on='userId', right_on='userId')

In [24]:
ratings.compute().isnull().sum()

userId                                   0
movieId                                  0
GT                                       0
timestamp                                0
user_rt_count                            0
user_rt_mean                             0
movie_rt_mean                            0
popularity                               0
weekday                                  0
weekday_6                                0
weekday_2                                0
weekday_3                                0
weekday_0                                0
weekday_1                                0
weekday_5                                0
weekday_4                                0
id                                       0
Action                                   0
Adventure                                0
Animation                                0
Aniplex                                  0
BROSTA TV                                0
Carousel Productions                     0
Comedy     

# Upload Users and Movies to MongoDB Database

In [ ]:
ratings_df = ratings.compute()
#Uploading movies to MongoDB cluster.
mdb.addMoviesBulk(ratings_df,users_genres.compute())
# Uploading users to MongoDB Cluster
mdb.addUsersbulk(ratings_df)

# __Supervised Learning__

## __Full Dataset__

### X & y definition

In [26]:
X_columns = ['user_rt_mean', 'movie_rt_mean', 'popularity', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War',
       'Western']
y_columns = ['GT']

In [27]:
X = ratings[X_columns].to_dask_array(lengths=True)
y = ratings[y_columns].to_dask_array(lengths=True)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y.ravel(),test_size=0.2)

### Model Selection

In [29]:
models = {
        "RandomForest": RandomForestRegressor(n_estimators=100, n_jobs=-1),
        "SGDRegressor": linear_model.SGDRegressor(max_iter=1000, tol=1e-3),
        "DecisionTree" : tree.DecisionTreeRegressor(random_state=0),
        "GradientBoostingRegressor": GradientBoostingRegressor(n_estimators=100),
        "AdaBoostRegressor" : AdaBoostRegressor(n_estimators=100)    
    }
with joblib.parallel_backend('dask'):
    %time _ = sl.mlmodelSelection(models, X_train, X_test, y_train, y_test)  


Training model: RandomForest


KeyboardInterrupt: 

In [ ]:
#GradientBoostingRegressor
#Hyperparameters search
model = GradientBoostingRegressor()
params = {
    'loss' : ['ls', 'lad', 'huber', 'quantile'],
    'n_estimators' : [100, 400, 700],
    'max_depth' : [3, 10, 15]
}

with joblib.parallel_backend('dask'):
    search = GridSearchCV(model, params)
    search.fit(X_train, y_train)

In [ ]:
pd.DataFrame(search.cv_results_)

In [30]:
#Final Training
with joblib.parallel_backend('dask'):
    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    print("RSME", (mean_squared_error(y_test, y_pred)**0.5))
    print("MAE", mean_absolute_error(y_test, y_pred))
    print("r2_score", r2_score(y_test, y_pred))

RSME 0.8213389587781282
MAE 0.6303051118003192
r2_score 0.3555883490606988


In [31]:
#saving the model to a pickle
with open('./output/models/gbrdefaultpickle_file.joblib', 'wb') as gbr_file:  
    joblib.dump(gbr, gbr_file)